# Common imports

In [ ]:
#%matplotlib inline
%matplotlib notebook
import os
import sys
import glob
import re
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from PIL import Image
import time

# Provided functions
### Image loading and saving

In [ ]:
def LoadFaceImages(pathname, subject_name, num_images):
    """
    Load the set of face images.  
    The routine returns
        ambimage: image illuminated under the ambient lighting
        imarray: a 3-D array of images, h x w x Nimages
        lightdirs: Nimages x 3 array of light source directions
    """

    def load_image(fname):
        return np.asarray(Image.open(fname))

    def fname_to_ang(fname):
        yale_name = os.path.basename(fname)
        return int(yale_name[12:16]), int(yale_name[17:20])

    def sph2cart(az, el, r):
        rcos_theta = r * np.cos(el)
        x = rcos_theta * np.cos(az)
        y = rcos_theta * np.sin(az)
        z = r * np.sin(el)
        return x, y, z

    ambimage = load_image(
        os.path.join(pathname, subject_name + '_P00_Ambient.pgm'))
    im_list = glob.glob(os.path.join(pathname, subject_name + '_P00A*.pgm'))
    if num_images <= len(im_list):
        im_sub_list = np.random.choice(im_list, num_images, replace=False)
    else:
        print(
            'Total available images is less than specified.\nProceeding with %d images.\n'
            % len(im_list))
        im_sub_list = im_list
    im_sub_list.sort()
    imarray = np.stack([load_image(fname) for fname in im_sub_list], axis=-1)
    Ang = np.array([fname_to_ang(fname) for fname in im_sub_list])

    x, y, z = sph2cart(Ang[:, 0] / 180.0 * np.pi, Ang[:, 1] / 180.0 * np.pi, 1)
    lightdirs = np.stack([y, z, x], axis=-1)
    return ambimage, imarray, lightdirs

In [ ]:
def save_outputs(subject_name, albedo_image, surface_normals):
    im = Image.fromarray((albedo_image*255).astype(np.uint8))
    im.save("%s_albedo.jpg" % subject_name)
    im = Image.fromarray((surface_normals[:,:,0]*128+128).astype(np.uint8))
    im.save("%s_normals_x.jpg" % subject_name)
    im = Image.fromarray((surface_normals[:,:,1]*128+128).astype(np.uint8))
    im.save("%s_normals_y.jpg" % subject_name)
    im = Image.fromarray((surface_normals[:,:,2]*128+128).astype(np.uint8))
    im.save("%s_normals_z.jpg" % subject_name)

### Plot the height map

In [ ]:
def set_aspect_equal_3d(ax):
    """https://stackoverflow.com/questions/13685386"""
    """Fix equal aspect bug for 3D plots."""
    xlim = ax.get_xlim3d()
    ylim = ax.get_ylim3d()
    zlim = ax.get_zlim3d()
    from numpy import mean
    xmean = mean(xlim)
    ymean = mean(ylim)
    zmean = mean(zlim)
    plot_radius = max([
        abs(lim - mean_)
        for lims, mean_ in ((xlim, xmean), (ylim, ymean), (zlim, zmean))
        for lim in lims
    ])
    ax.set_xlim3d([xmean - plot_radius, xmean + plot_radius])
    ax.set_ylim3d([ymean - plot_radius, ymean + plot_radius])
    ax.set_zlim3d([zmean - plot_radius, zmean + plot_radius])


def display_output(albedo_image, height_map):
    fig = plt.figure()
    plt.imshow(albedo_image, cmap='gray')
    plt.axis('off')
    
    fig = plt.figure(figsize=(10, 10))
    ax = fig.gca(projection='3d')
    ax.view_init(20, 20)
    X = np.arange(albedo_image.shape[0])
    Y = np.arange(albedo_image.shape[1])
    X, Y = np.meshgrid(Y, X)
    H = np.flipud(np.fliplr(height_map))
    A = np.flipud(np.fliplr(albedo_image))
    A = np.stack([A, A, A], axis=-1)
    ax.xaxis.set_ticks([])
    ax.xaxis.set_label_text('Z')
    ax.yaxis.set_ticks([])
    ax.yaxis.set_label_text('X')
    ax.zaxis.set_ticks([])
    ax.yaxis.set_label_text('Y')
    surf = ax.plot_surface(
        H, X, Y, cmap='gray', facecolors=A, linewidth=0, antialiased=False)
    set_aspect_equal_3d(ax)

### Plot the surface norms. 

In [ ]:
def plot_surface_normals(surface_normals):
    """
    surface_normals: h x w x 3 matrix.
    """
    fig = plt.figure()
    ax = plt.subplot(1, 3, 1)
    ax.axis('off')
    ax.set_title('X')
    im = ax.imshow(surface_normals[:,:,0])
    ax = plt.subplot(1, 3, 2)
    ax.axis('off')
    ax.set_title('Y')
    im = ax.imshow(surface_normals[:,:,1])
    ax = plt.subplot(1, 3, 3)
    ax.axis('off')
    ax.set_title('Z')
    im = ax.imshow(surface_normals[:,:,2])

# Your implementation

In [ ]:
def preprocess(ambimage, imarray):
    """
    preprocess the data: 
        1. subtract ambient_image from each image in imarray.
        2. make sure no pixel is less than zero.
        3. rescale values in imarray to be between 0 and 1.
    Inputs:
        ambimage: h x w
        imarray: h x w x Nimages
    Outputs:
        processed_imarray: h x w x Nimages
    """
    copied_ambimage = np.asarray(ambimage).astype(np.float64)
    copied_imarray = np.asarray(imarray).astype(np.float64)
    # subtract
    subtracted = copied_imarray - copied_ambimage[:,:,None]

    # no pixel is less than zero
    clipped = np.clip(subtracted, a_min = 0, a_max = None)
    
    # rescale
    processed_imarray = clipped / 255
    
    return processed_imarray

In [ ]:
def photometric_stereo(imarray, light_dirs):
    """
    Inputs:
        imarray:  h x w x Nimages
        light_dirs: Nimages x 3
    Outputs:
        albedo_image: h x w
        surface_norms: h x w x 3
    """
    # shape h, w, N
    copied_imarray = np.asarray(imarray).astype(np.float64)
    
    # shape N, h x w
    transformed_imarray = copied_imarray.reshape(copied_imarray.shape[0] * copied_imarray.shape[1], copied_imarray.shape[2]).transpose()
    
    # print(transformed_imarray.shape)
    
    g, residuals, rank, s = np.linalg.lstsq(light_dirs, transformed_imarray, rcond=None)
    # print(g.shape)
    
    g_new = g.transpose().reshape(192, 168, 3)
    # print(g_new.shape)
    
    albedo_image = np.linalg.norm(g_new, axis=2)
    
    surface_normals = g_new / albedo_image[:,:,None]
    
    return albedo_image, surface_normals

In [ ]:
def get_surface(surface_normals, integration_method):
    """
    Inputs:
        surface_normals:h x w x 3
        integration_method: string in ['average', 'column', 'row', 'random']
    Outputs:
        height_map: h x w
    """
    g1_xy = surface_normals[:, :, 0]
    
    g2_xy = surface_normals[:, :, 1]
    
    g3_xy = surface_normals[:, :, 2]
    
    fx_xy = g1_xy / g3_xy
    
    fy_xy = g2_xy / g3_xy
    
    if integration_method == 'row':
        height_map = np.cumsum(fy_xy, axis=0) + np.cumsum(fx_xy, axis = 1)[0]
        
    if integration_method == 'column':
        height_map = (np.cumsum(fy_xy, axis=0)[:, 0])[:, None] + np.cumsum(fx_xy, axis = 1)
    
    if integration_method == 'average':
        height_map1 = np.cumsum(fy_xy, axis=0) + np.cumsum(fx_xy, axis = 1)[0]
        height_map2 = (np.cumsum(fy_xy, axis=0)[:, 0])[:, None] + np.cumsum(fx_xy, axis = 1)
        height_map = (height_map1 + height_map2) / 2
 


    if integration_method == 'random':
        iteration_num = 10000
        height_map = np.zeros((192, 168))
        
        for i in range(iteration_num):
            random_row = np.random.randint(10, 182)
            random_col = np.random.randint(10, 159)
            
            
            # lu 
            fx_xy_lu = fx_xy[0 : random_row + 1, 0 : random_col + 1]
            fy_xy_lu = fy_xy[0 : random_row + 1, 0 : random_col + 1]
                
            fx_xy_lu_flipped = np.flip(fx_xy_lu, axis = 0)
            fx_xy_lu_flipped = np.flip(fx_xy_lu_flipped, axis = 1)
            
            fy_xy_lu_flipped = np.flip(fy_xy_lu, axis = 0)
            fy_xy_lu_flipped = np.flip(fy_xy_lu_flipped, axis = 1)
            
            
            # ru
            fx_xy_ru = fx_xy[0 : random_row, random_col + 1 : 169]
            fy_xy_ru = fy_xy[0 : random_row, random_col + 1 : 169]
                
            fx_xy_ru_flipped = np.flip(fx_xy_ru, axis = 0)
            fy_xy_ru_flipped = np.flip(fy_xy_ru, axis = 0)
                
                
            #lb
            fx_xy_lb = fx_xy[random_row + 1 : 193, 0 : random_col]
            fy_xy_lb = fy_xy[random_row + 1 : 193, 0 : random_col]
                
            fx_xy_lb_flipped = np.flip(fx_xy_lb, axis = 1)
            fy_xy_lb_flipped = np.flip(fy_xy_lb, axis = 1)            
            
            
            # rb
            fx_xy_rb = fx_xy[random_row : 193, random_col : 169]
            fy_xy_rb = fy_xy[random_row : 193, random_col : 169]
            

            
            integration_method = np.random.randint(2)
            # row first
            if integration_method == 0:
                # lu
                lu_result = np.cumsum(fx_xy_lu_flipped, axis=1)[0] + np.cumsum(fy_xy_lu_flipped, axis = 0)
                lu_result = np.flip(lu_result, axis = 0)
                lu_result = np.flip(lu_result, axis = 1)
                height_map[0 : random_row + 1, 0 : random_col + 1] -= lu_result
                
                # ru
                ru_result = np.cumsum(fx_xy_ru_flipped, axis = 1)[0] - np.cumsum(fy_xy_ru_flipped, axis=0)
                ru_result = np.flip(ru_result, axis = 0)
                height_map[0 : random_row, random_col + 1 : 169] += ru_result
                
                # lb
                lb_result = -np.cumsum(fx_xy_lb_flipped, axis = 1)[0] + np.cumsum(fy_xy_lb_flipped, axis=0)
                lb_result = np.flip(lb_result, axis = 1)
                height_map[random_row + 1 : 193, 0 : random_col] += lb_result
                
                # rb
                rb_result = np.cumsum(fx_xy_rb, axis=1)[0] + np.cumsum(fy_xy_rb, axis = 0)
                height_map[random_row : 193, random_col : 169] += rb_result
                
            # column first
            elif integration_method == 1:
                # lu
                lu_result = (np.cumsum(fy_xy_lu_flipped, axis=0)[:, 0])[:, None] + np.cumsum(fx_xy_lu_flipped, axis = 1)
                lu_result = np.flip(lu_result, axis = 0)
                lu_result = np.flip(lu_result, axis = 1)
                height_map[0 : random_row + 1, 0 : random_col + 1] -= lu_result
                
                # ru
                ru_result = -(np.cumsum(fy_xy_ru_flipped, axis=0)[:, 0])[:, None] + np.cumsum(fx_xy_ru_flipped, axis = 1)
                ru_result = np.flip(ru_result, axis = 0)
                height_map[0 : random_row, random_col + 1 : 169] += ru_result
                
                # lb
                lb_result = (np.cumsum(fy_xy_lb_flipped, axis=0)[:, 0])[:, None] - np.cumsum(fx_xy_lb_flipped, axis = 1)
                lb_result = np.flip(lb_result, axis = 1)
                height_map[random_row + 1 : 193, 0 : random_col] += lb_result
                
                # rb
                rb_result = (np.cumsum(fy_xy_rb, axis=0)[:, 0])[:, None] + np.cumsum(fx_xy_rb, axis = 1)
                height_map[random_row : 193, random_col : 169] += rb_result
                
                
        height_map  = height_map - height_map[0, 0]
        height_map = height_map / iteration_num

        
    return height_map

# Main function

### 01, row

In [ ]:
# Given
root_path = '../croppedyale/'
subject_name = 'yaleB01'
integration_method = 'row'
save_flag = True

full_path = '%s%s' % (root_path, subject_name)
ambient_image, imarray, light_dirs = LoadFaceImages(full_path, subject_name, 64)

processed_imarray = preprocess(ambient_image, imarray)

albedo_image, surface_normals = photometric_stereo(processed_imarray, light_dirs)
tic = time.clock()
height_map = get_surface(surface_normals, integration_method)
toc = time.clock()
print(toc - tic)
if save_flag:
    save_outputs(subject_name, albedo_image, surface_normals)

In [ ]:
# Given
plot_surface_normals(surface_normals)

In [ ]:
# Given
display_output(albedo_image, height_map)

### 01, column

In [ ]:

root_path = '../croppedyale/'
subject_name = 'yaleB01'
integration_method = 'column'
save_flag = False

full_path = '%s%s' % (root_path, subject_name)
ambient_image, imarray, light_dirs = LoadFaceImages(full_path, subject_name, 64)

processed_imarray = preprocess(ambient_image, imarray)

albedo_image, surface_normals = photometric_stereo(processed_imarray, light_dirs)
tic = time.clock()
height_map = get_surface(surface_normals, integration_method)
toc = time.clock()
print(toc - tic)

if save_flag:
    save_outputs(subject_name, albedo_image, surface_normals)

In [ ]:
plot_surface_normals(surface_normals)

In [ ]:
display_output(albedo_image, height_map)

### 01, average

In [ ]:

root_path = '../croppedyale/'
subject_name = 'yaleB01'
integration_method = 'average'
save_flag = False

full_path = '%s%s' % (root_path, subject_name)
ambient_image, imarray, light_dirs = LoadFaceImages(full_path, subject_name, 64)

processed_imarray = preprocess(ambient_image, imarray)

albedo_image, surface_normals = photometric_stereo(processed_imarray, light_dirs)

tic = time.clock()
height_map = get_surface(surface_normals, integration_method)
toc = time.clock()
print(toc - tic)
if save_flag:
    save_outputs(subject_name, albedo_image, surface_normals)

In [ ]:
plot_surface_normals(surface_normals)

In [ ]:
display_output(albedo_image, height_map)

### 01, random

In [ ]:

root_path = '../croppedyale/'
subject_name = 'yaleB01'
integration_method = 'random'
save_flag = False

full_path = '%s%s' % (root_path, subject_name)
ambient_image, imarray, light_dirs = LoadFaceImages(full_path, subject_name, 64)

processed_imarray = preprocess(ambient_image, imarray)

albedo_image, surface_normals = photometric_stereo(processed_imarray, light_dirs)
tic = time.clock()
height_map = get_surface(surface_normals, integration_method)
toc = time.clock()
print(toc - tic)
if save_flag:
    save_outputs(subject_name, albedo_image, surface_normals)

In [ ]:
plot_surface_normals(surface_normals)

In [ ]:
display_output(albedo_image, height_map)

### 02, row

In [ ]:
root_path = '../croppedyale/'
subject_name = 'yaleB02'
integration_method = 'row'
save_flag = True

full_path = '%s%s' % (root_path, subject_name)
ambient_image, imarray, light_dirs = LoadFaceImages(full_path, subject_name, 64)

processed_imarray = preprocess(ambient_image, imarray)

albedo_image, surface_normals = photometric_stereo(processed_imarray, light_dirs)

height_map = get_surface(surface_normals, integration_method)

if save_flag:
    save_outputs(subject_name, albedo_image, surface_normals)

In [ ]:
plot_surface_normals(surface_normals)

In [ ]:
display_output(albedo_image, height_map)

### 02, column

In [ ]:
root_path = '../croppedyale/'
subject_name = 'yaleB02'
integration_method = 'column'
save_flag = False

full_path = '%s%s' % (root_path, subject_name)
ambient_image, imarray, light_dirs = LoadFaceImages(full_path, subject_name, 64)

processed_imarray = preprocess(ambient_image, imarray)

albedo_image, surface_normals = photometric_stereo(processed_imarray, light_dirs)

height_map = get_surface(surface_normals, integration_method)

if save_flag:
    save_outputs(subject_name, albedo_image, surface_normals)

In [ ]:
plot_surface_normals(surface_normals)

In [ ]:
display_output(albedo_image, height_map)

### 02, average

In [ ]:
root_path = '../croppedyale/'
subject_name = 'yaleB02'
integration_method = 'average'
save_flag = False

full_path = '%s%s' % (root_path, subject_name)
ambient_image, imarray, light_dirs = LoadFaceImages(full_path, subject_name, 64)

processed_imarray = preprocess(ambient_image, imarray)

albedo_image, surface_normals = photometric_stereo(processed_imarray, light_dirs)

height_map = get_surface(surface_normals, integration_method)

if save_flag:
    save_outputs(subject_name, albedo_image, surface_normals)

In [ ]:
plot_surface_normals(surface_normals)

In [ ]:
display_output(albedo_image, height_map)

### 02, random

In [ ]:
root_path = '../croppedyale/'
subject_name = 'yaleB02'
integration_method = 'random'
save_flag = False

full_path = '%s%s' % (root_path, subject_name)
ambient_image, imarray, light_dirs = LoadFaceImages(full_path, subject_name, 64)

processed_imarray = preprocess(ambient_image, imarray)

albedo_image, surface_normals = photometric_stereo(processed_imarray, light_dirs)

height_map = get_surface(surface_normals, integration_method)

if save_flag:
    save_outputs(subject_name, albedo_image, surface_normals)

In [ ]:
plot_surface_normals(surface_normals)

In [ ]:
display_output(albedo_image, height_map)

### 05, row

In [ ]:
root_path = '../croppedyale/'
subject_name = 'yaleB05'
integration_method = 'row'
save_flag = True

full_path = '%s%s' % (root_path, subject_name)
ambient_image, imarray, light_dirs = LoadFaceImages(full_path, subject_name, 64)

processed_imarray = preprocess(ambient_image, imarray)

albedo_image, surface_normals = photometric_stereo(processed_imarray, light_dirs)

height_map = get_surface(surface_normals, integration_method)

if save_flag:
    save_outputs(subject_name, albedo_image, surface_normals)

In [ ]:
plot_surface_normals(surface_normals)

In [ ]:
display_output(albedo_image, height_map)

### 05, column

In [ ]:
root_path = '../croppedyale/'
subject_name = 'yaleB05'
integration_method = 'column'
save_flag = False

full_path = '%s%s' % (root_path, subject_name)
ambient_image, imarray, light_dirs = LoadFaceImages(full_path, subject_name, 64)

processed_imarray = preprocess(ambient_image, imarray)

albedo_image, surface_normals = photometric_stereo(processed_imarray, light_dirs)

height_map = get_surface(surface_normals, integration_method)

if save_flag:
    save_outputs(subject_name, albedo_image, surface_normals)

In [ ]:
plot_surface_normals(surface_normals)

In [ ]:
display_output(albedo_image, height_map)

### 05, average

In [ ]:
root_path = '../croppedyale/'
subject_name = 'yaleB05'
integration_method = 'average'
save_flag = False

full_path = '%s%s' % (root_path, subject_name)
ambient_image, imarray, light_dirs = LoadFaceImages(full_path, subject_name, 64)

processed_imarray = preprocess(ambient_image, imarray)

albedo_image, surface_normals = photometric_stereo(processed_imarray, light_dirs)

height_map = get_surface(surface_normals, integration_method)

if save_flag:
    save_outputs(subject_name, albedo_image, surface_normals)

In [ ]:
plot_surface_normals(surface_normals)

In [ ]:
display_output(albedo_image, height_map)

### 05, random

In [ ]:
root_path = '../croppedyale/'
subject_name = 'yaleB05'
integration_method = 'random'
save_flag = False

full_path = '%s%s' % (root_path, subject_name)
ambient_image, imarray, light_dirs = LoadFaceImages(full_path, subject_name, 64)

processed_imarray = preprocess(ambient_image, imarray)

albedo_image, surface_normals = photometric_stereo(processed_imarray, light_dirs)

height_map = get_surface(surface_normals, integration_method)

if save_flag:
    save_outputs(subject_name, albedo_image, surface_normals)

In [ ]:
plot_surface_normals(surface_normals)

In [ ]:
display_output(albedo_image, height_map)

### 07, row

In [ ]:
root_path = '../croppedyale/'
subject_name = 'yaleB07'
integration_method = 'row'
save_flag = True

full_path = '%s%s' % (root_path, subject_name)
ambient_image, imarray, light_dirs = LoadFaceImages(full_path, subject_name, 64)

processed_imarray = preprocess(ambient_image, imarray)

albedo_image, surface_normals = photometric_stereo(processed_imarray, light_dirs)

height_map = get_surface(surface_normals, integration_method)

if save_flag:
    save_outputs(subject_name, albedo_image, surface_normals)

In [ ]:
plot_surface_normals(surface_normals)

In [ ]:
display_output(albedo_image, height_map)

### 07, column

In [ ]:
root_path = '../croppedyale/'
subject_name = 'yaleB07'
integration_method = 'column'
save_flag = False

full_path = '%s%s' % (root_path, subject_name)
ambient_image, imarray, light_dirs = LoadFaceImages(full_path, subject_name, 64)

processed_imarray = preprocess(ambient_image, imarray)

albedo_image, surface_normals = photometric_stereo(processed_imarray, light_dirs)

height_map = get_surface(surface_normals, integration_method)

if save_flag:
    save_outputs(subject_name, albedo_image, surface_normals)

In [ ]:
plot_surface_normals(surface_normals)

In [ ]:
display_output(albedo_image, height_map)

### 07, average

In [ ]:
root_path = '../croppedyale/'
subject_name = 'yaleB07'
integration_method = 'average'
save_flag = False

full_path = '%s%s' % (root_path, subject_name)
ambient_image, imarray, light_dirs = LoadFaceImages(full_path, subject_name, 64)

processed_imarray = preprocess(ambient_image, imarray)

albedo_image, surface_normals = photometric_stereo(processed_imarray, light_dirs)

height_map = get_surface(surface_normals, integration_method)

if save_flag:
    save_outputs(subject_name, albedo_image, surface_normals)

In [ ]:
plot_surface_normals(surface_normals)

In [ ]:
display_output(albedo_image, height_map)

### 07, random

In [ ]:
root_path = '../croppedyale/'
subject_name = 'yaleB07'
integration_method = 'random'
save_flag = False

full_path = '%s%s' % (root_path, subject_name)
ambient_image, imarray, light_dirs = LoadFaceImages(full_path, subject_name, 64)

processed_imarray = preprocess(ambient_image, imarray)

albedo_image, surface_normals = photometric_stereo(processed_imarray, light_dirs)

height_map = get_surface(surface_normals, integration_method)

if save_flag:
    save_outputs(subject_name, albedo_image, surface_normals)

In [ ]:
plot_surface_normals(surface_normals)

In [ ]:
display_output(albedo_image, height_map)

### Test

In [ ]:
root_path = '../'
subject_name = 'yaleB01'
integration_method = 'random'
save_flag = False

full_path = '%s%s' % (root_path, subject_name)
ambient_image, imarray, light_dirs = LoadFaceImages(full_path, subject_name, 25)

processed_imarray = preprocess(ambient_image, imarray)

albedo_image, surface_normals = photometric_stereo(processed_imarray, light_dirs)

height_map = get_surface(surface_normals, integration_method)

if save_flag:
    save_outputs(subject_name, albedo_image, surface_normals)

In [ ]:
plot_surface_normals(surface_normals)

In [ ]:
display_output(albedo_image, height_map)